# Big data workbook #2

We start by getting set up

In [ ]:
import pandas as pd
import altair as alt
import json

Next, we have a look at the dataset

In [ ]:
prices_df = pd.read_parquet('https://autocpi-public.s3.eu-west-2.amazonaws.com/lrpd/db_prices.parquet')
prices_df.describe()

,quote_date,shop_code,item_id_raw,region,price,item_id
count,4.836896e+07,4.836896e+07,4.836897e+07,4.836896e+07,4.836896e+07,4.836897e+07
mean,2.007762e+05,4.771270e+02,3.880409e+05,6.679112e+00,4.960397e+01,3.883983e+05
std,1.060226e+03,1.531775e+03,1.467557e+05,3.407499e+00,2.058161e+02,1.466723e+05
min,1.988020e+05,1.000000e+00,2.101010e+05,1.000000e+00,1.000000e-02,2.101010e+05
25%,1.998110e+05,3.900000e+01,2.129170e+05,3.000000e+00,1.490000e+00,2.129180e+05
50%,2.008050e+05,8.800000e+01,4.301280e+05,7.000000e+00,4.850000e+00,4.301320e+05
75%,2.017070e+05,8.020000e+02,5.104060e+05,9.000000e+00,1.999000e+01,5.104070e+05
max,2.025100e+05,2.007100e+04,6.404060e+05,1.300000e+01,4.400000e+04,6.404060e+05


Read the items into our key

In [ ]:
items_df = pd.read_parquet('https://autocpi-public.s3.eu-west-2.amazonaws.com/lrpd/db_item.parquet')
items_df.head()

,item_id,description,date_quote_s,date_quote_e,n_obs
0,210101,LARGE LOAF-WHITE-SLICED-800G,198802,200401,36039
1,210102,LARGE LOAF-WHITE-UNSLICED-800G,198802,202510,56917
2,210105,LARGE WHOLEMEAL LOAF-UNSLICED,198802,200301,27161
3,210106,SIX BREAD ROLLS-WHITE/BROWN,198802,202510,67469
4,210107,"BROWN LOAF,400G,SLICED-GRAN",198903,200401,29361


Look for the item id for olive oil

In [ ]:
items_df[items_df['description'].str.contains('oil', case=False)]

,item_id,description,date_quote_s,date_quote_e,n_obs
134,211404,COOKING OIL -VEGETABLE 1 LITRE,198802,200612,37627
137,211408,OLIVE OIL - 500ML - 1 LITRE,200702,202510,50000
141,211412,SPRAY OIL BOTTLE 190-220MLS,202402,202510,3599
725,430505,ALUMIN COOKING FOIL 290-300MM,198802,202510,68496
728,430510,TOILET ROLLS (PACKET OF 2),198802,200501,36662
737,430521,TOILET ROLLS- 8 TO 10 PACK,199402,200401,14845
752,430536,TOILET ROLLS,200502,202510,37064
1086,520208,TOILET SOAP-LARGE BAR,198802,200401,29714
1100,520225,TOILET SOAP-LARGE BAR-3/4 PCK,199402,200001,8235
1115,520240,BAR OF TOILET SOAP (100-125G),200402,201001,10876


Find the mean, median, and mode for item id 211408

In [ ]:
EVOO_price = prices_df.query("item_id == 211408")
avg_EVOO_price = EVOO_price.groupby('quote_date').agg(
    price_mean=('price', 'mean'),
    price_median=('price', 'median'),
    price_mode=('price', lambda x: x.mode()[0] if not x.mode().empty else None)
).reset_index()

avg_EVOO_price.columns = ['date', 'Mean', 'Median', 'Mode']
avg_EVOO_price

,date,Mean,Median,Mode
0,200702.0,4.061088,3.99,4.15
1,200703.0,4.101741,3.99,4.15
2,200704.0,3.988241,3.99,4.15
3,200705.0,3.968039,3.99,4.15
4,200706.0,3.933582,3.92,4.15
...,...,...,...,...
220,202506.0,7.523702,7.00,5.55
221,202507.0,7.282092,6.49,5.55
222,202508.0,7.387469,6.80,5.55
223,202509.0,7.093892,6.50,5.55


Melt the data down so its compatible with vegalite

In [ ]:
avg_EVOO_price_melted = avg_EVOO_price.melt(id_vars=['date'],
                                                   var_name='price_type',
                                                   value_name='price')

avg_EVOO_price_melted['date'] = pd.to_datetime(avg_EVOO_price_melted['date'], format='%Y%m')

avg_EVOO_price_melted

,date,price_type,price
0,2007-02-01,Mean,4.061088
1,2007-03-01,Mean,4.101741
2,2007-04-01,Mean,3.988241
3,2007-05-01,Mean,3.968039
4,2007-06-01,Mean,3.933582
...,...,...,...
670,2025-06-01,Mode,5.550000
671,2025-07-01,Mode,5.550000
672,2025-08-01,Mode,5.550000
673,2025-09-01,Mode,5.550000


Mock up the chart

In [ ]:
chart = (
    alt.Chart(avg_EVOO_price_melted)
    .mark_line()
    .encode(
        x=alt.X('date:T', title=''),
        y=alt.Y('price:Q', title='Price (GBP)'),
        color='price_type:N'
    )
    .properties(
        title={
            "text": "Olive Oil Price",
            "subtitle": [
                "Mean, median, and mode price for olive oil",
                "Source: ONS microdata via Davies (2021)"
            ],
            "fontSize": 16
        }
    )
)

spec = chart.to_dict()

with open('olive_oil_price_chart.json', 'w') as f:
    json.dump(spec, f, indent=2)